In [1]:
#Imports
import numpy as np
import pandas as pd

import json
from gensim.models import Word2Vec


from IPython.core.display import display, HTML

import HTMLSentenceTokenizer
sentence = HTMLSentenceTokenizer.HTMLSentenceTokenizer()
from nltk.tokenize import word_tokenize
import ast

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")


import functions 
clean = functions.clean()

In [2]:
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/{}"

In [3]:
att = pd.read_csv("/Users/conorosully/Documents/Legal-Case-Prediction/data/preclean/case_attributes.csv",'#')
print(att.columns)
print(len(att))
IDs = att['id']
att.head(1)

Index(['id', 'type', 'url', 'doc-name', 'doc-id-label', 'doc-date', 'vl-type',
       'created-at', 'updated-at', 'app-no-parts', 'app-nos', 'articles',
       'conclusions', 'courts', 'ecli', 'judgement-date', 'judges',
       'last-modified-time', 'nonviolations', 'organisations', 'respondents',
       'separate-opinion', 'violations'],
      dtype='object')
55652


,id,type,url,doc-name,doc-id-label,doc-date,vl-type,created-at,updated-at,app-no-parts,...,courts,ecli,judgement-date,judges,last-modified-time,nonviolations,organisations,respondents,separate-opinion,violations
0,HUDOC-ECHR-1959-001-49198,eu-chrs,http://hudoc.echr.coe.int/app/conversion/docx/...,greece <br>against<br> the united kingdom,176/56,1959-04-20,eu_chr,2017-09-18T10:01:36.112Z,2018-10-26T16:53:58.954Z,"['176', '56']",...,NaN,NaN,NaN,NaN,2018-10-07,NaN,['Committee of Ministers'],['GBR'],NaN,NaN


In [4]:
ID = 'HUDOC-ECHR-1999-001-58225'
doc = clean.getCase(ID)[ID]
print(type(doc))

#clean.renderHTML(ID)

<class 'str'>


### Create sentences

In [7]:
sentences = []

count = 1
for ID in IDs:
    doc = clean.getCase(ID)[ID]
    parsed_sentences = sentence.feed(doc)
    for sent in parsed_sentences:
        tokens = word_tokenize(sent)

        tokens = [word for word in tokens if word.isalpha()]
        tokens = [w.lower() for w in tokens]
        if len(tokens) > 2:
            sentences.append(tokens)
    
    count += 1
    if count%5000 == 0:
        print('Processed {}/55652'.format(count))
              
print(len(sentences))
print(sentences[0])

Processed 5000/55652
Processed 10000/55652
Processed 15000/55652
Processed 20000/55652
Processed 25000/55652
Processed 30000/55652
Processed 35000/55652
Processed 40000/55652
Processed 45000/55652
Processed 50000/55652
Processed 55000/55652
4824978
['the', 'committee', 'of', 'ministers']


In [8]:
"""with open(path.format('preclean/sentences.txt'), 'w') as f:
    for s in sentences:
        f.write(str(s) + '\n')"""

### Create embeddings

In [9]:
with open(path.format('preclean/sentences.txt'), 'r') as f:
    sentences = [line.rstrip('\n') for line in f]
sentences = [ast.literal_eval(s) for s in sentences ]
print(len(sentences))
print(sentences[0])

4824978
['the', 'committee', 'of', 'ministers']


In [13]:
count = 0
for s in tqdm(sentences):
    count += len(s)
print(count)

100%|██████████| 4824978/4824978 [00:02<00:00, 1785040.89it/s]

84007186


In [18]:
sentences[0][0]

'the'

In [14]:
# define training data
size = 100
for s in [100,200]:
    # train model
    model = Word2Vec(sentences, min_count=10,size=s)

    words = list(model.wv.vocab)
    #print(model['case'])
    name = 'embeddings/echt2vec_{}.txt'.format(s)
    model.save(path.format(name))
    print(s)

/Users/conorosully/miniconda3/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


100
200


In [15]:
new_model = Word2Vec.load(path.format('embeddings/echt2vec_100.txt'))
print(new_model)
print(new_model['the'])

Word2Vec(vocab=47597, size=100, alpha=0.025)
[-1.5897316   1.1752322   0.1812835  -2.0753927  -0.30365354 -2.623864
 -3.0174785  -0.53128755  1.0208697  -0.7977395   0.49885014  1.9134829
 -0.9142087   0.3425168  -0.93032765  1.6869735  -2.96303    -0.1425623
  0.87882996 -2.6694467   0.7219321  -1.5780528  -1.5487238   1.2211422
  1.6357045  -0.43606836  0.92221355  0.52769876 -0.10625964  0.54692
 -2.2359977  -0.23140007 -0.10876626 -2.17795    -1.6003083  -1.1905388
 -1.2994738   0.27796122  0.6485265   0.4778275  -0.5858192  -1.9545654
 -2.2794394  -0.706583   -0.27957568 -1.0418987  -2.3569665  -2.870128
  2.234844    0.980896   -0.23302387 -0.9976439  -0.3800027   0.41793412
  1.8830599   0.04513555 -0.3611936  -3.750025    1.0965487   0.40400803
  2.7103674  -0.08127059  1.2497718   0.7829084   0.7177596  -0.964606
  0.67719966  1.6630825   2.7763338  -0.88545233 -0.28383926 -0.07257998
 -1.5132622  -0.33038953  0.08622823  1.3757161  -0.80610454 -0.45928687
 -0.5744083   0.5190

/Users/conorosully/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
